<img src="https://raw.githubusercontent.com/maxsitt/insect-detect-docs/main/docs/assets/logo.png" width="500">

# YOLOv5 detection model training + conversion for deployment on Luxonis OAK

> Compiled by: &nbsp; **Maximilian Sittinger** &nbsp;
[<img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width="24">](https://github.com/maxsitt) &nbsp;
[<img src="https://upload.wikimedia.org/wikipedia/commons/0/06/ORCID_iD.svg" width="24">](https://orcid.org/0000-0002-4096-8556)  

- [`insect-detect-ml` GitHub repo](https://github.com/maxsitt/insect-detect-ml)
- 📑 [**Insect Detect Docs**](https://maxsitt.github.io/insect-detect-docs/)

&nbsp;

**This notebook will enable you to train a [YOLOv5](https://github.com/ultralytics/yolov5) object detection model on your own custom training data.**

- Using dataset import from [Roboflow](https://roboflow.com/) is recommended, but is not required.
> Choose option *Upload dataset from Google Drive/local file system* instead (slower!).  
- Connecting to Google Drive is recommended, but is not required.
> Choose options *Upload dataset from your local file system* and *Download* instead of *Export to Google Drive* (slower!).  
- If you are using Firefox, please make sure to allow notifications for this website.

&nbsp;

---

**References**

1. Official YOLOv5 tutorial notebook by Ultralytics &nbsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb)
1. Roboflow tutorial notebook for YOLOv5 training &nbsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolov5-object-detection-on-custom-data.ipynb)
1. DepthAI tutorial notebook + conversion to .blob &nbsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/luxonis/depthai-ml-training/blob/master/colab-notebooks/YoloV5_training.ipynb)

# Initialization

## Show GPU + Linux distribution

In [ ]:
!nvidia-smi -L
print("\n")
!head -n 2 /etc/*release

## YOLOv5 setup

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import utils
display = utils.notebook_init()

## Recommended: Upload dataset from Roboflow

If you are not sure how to export your annotated dataset in YOLOv5 format, check the [Roboflow docs](https://docs.roboflow.com/exporting-data).

> Alternatively you can upload your dataset ([YOLOv5 format](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#1-create-dataset)) from **[Google Drive](#scrollTo=RxOnnOadc5vR)** or directly from your **[local file system](#scrollTo=qKTCWdtkOUw7)** in the next steps!

In [ ]:
%pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format = "yolov5", notebook = "insdet_yolov5")

**Copy only the last three lines of the Download Code and insert them at the top of the next code cell:**

In [ ]:
%cd /content/yolov5

### Paste your Download Code here:
rf = Roboflow(api_key="XXXXXXXXXXXXXXXXXXXXX")
project = rf.workspace("maximilian-sittinger").project("insect_detect_detection")
dataset = project.version(4).download("yolov5")
###

dataset_location = dataset.location

from pathlib import Path
print("\n")
print(f"Location of dataset: {dataset_location}")
print("\n")
print("Number of training images:", len(list(Path(f"{dataset_location}/train/images").glob("*.jpg"))))
print("Number of validation images:", len(list(Path(f"{dataset_location}/valid/images").glob("*.jpg"))))
print("Number of test images:", len(list(Path(f"{dataset_location}/test/images").glob("*.jpg"))))
print("\n")
%cat {dataset_location}/data.yaml

## Recommended: Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#@title ## Optional: Upload dataset from Google Drive {display-mode: "form"}

#@markdown ### Google Drive path to dataset folder:
dataset_path = "MyDrive/Datasets/yolov5_dataset" #@param {type: "string"}

%cp -ai /content/drive/{dataset_path} /content/yolov5

from pathlib import Path
dataset_name = Path(dataset_path).stem
dataset_location = f"/content/yolov5/{dataset_name}"

print(f"Location of dataset: {dataset_location}")
print("\n")
print("Number of training images:", len(list(Path(f"{dataset_location}/train/images").glob("*.jpg"))))
print("Number of validation images:", len(list(Path(f"{dataset_location}/valid/images").glob("*.jpg"))))
print("Number of test images:", len(list(Path(f"{dataset_location}/test/images").glob("*.jpg"))))
print("\n")
%cat {dataset_location}/data.yaml

In [ ]:
#@title ## Optional: Upload dataset from your local file system {display-mode: "form"}

#@markdown ### Name of your (zipped) dataset folder:
#@markdown Please make sure to compress your dataset folder to zip file before uploading!
dataset_name = "yolov5_dataset" #@param {type: "string"}
dataset_location = f"/content/yolov5/{dataset_name}"

from google.colab import files
uploaded = files.upload()

!unzip -uq {dataset_name}.zip -d /content/yolov5/
%rm {dataset_name}.zip

from pathlib import Path
print("\n")
print(f"Location of dataset: {dataset_location}")
print("\n")
print("Number of training images:", len(list(Path(f"{dataset_location}/train/images").glob("*.jpg"))))
print("Number of validation images:", len(list(Path(f"{dataset_location}/valid/images").glob("*.jpg"))))
print("Number of test images:", len(list(Path(f"{dataset_location}/test/images").glob("*.jpg"))))
print("\n")
%cat {dataset_location}/data.yaml

## Optional: Edit *data.yaml*
If you chose to upload your training dataset from **Google Drive** or your **local file system**, you have to edit the `data.yaml` file in your dataset folder to make sure the paths to the train, valid and test folders are correct.

Navigate to */content/yolov5* in the File Explorer (Folder symbol on the left side bar) and open your dataset folder. Double-click on the `data.yaml` file, it will open in the editor to the right. Make sure that the paths to the train, valid and test folders are as follows:

```
train: /content/yolov5/[YOUR_DATASET_NAME]/train/images
val: /content/yolov5/[YOUR_DATASET_NAME]/valid/images
test: /content/yolov5/[YOUR_DATASET_NAME]/test/images
```

* Insert the correct name of your dataset folder at `[YOUR_DATASET_NAME]`.
* Save your changes with **Ctrl + S** and close the editor.

# Model training

In [ ]:
#@title ## Optional: Select external logger {display-mode: "form"}

logger = "Weights&Biases" #@param ["Weights&Biases", "Comet", "ClearML"]

#@markdown **More info:** \
#@markdown - [Weights & Biases](https://github.com/ultralytics/yolov5/tree/master/utils/loggers/wandb)
#@markdown - [Comet](https://github.com/ultralytics/yolov5/tree/master/utils/loggers/comet)
#@markdown - [ClearML](https://github.com/ultralytics/yolov5/tree/master/utils/loggers/clearml)

if logger == "Weights&Biases":
  %pip install -q wandb
  import wandb
  wandb.login()
elif logger == "Comet":
  %pip install -q comet_ml
  import comet_ml
  comet_ml.init()
elif logger == "ClearML":
  %pip install -q clearml
  import clearml
  clearml.browser_login()

## Tensorboard logger

> If you are using Firefox, **disable Enhanced Tracking Protection** for this website (click on the shield to the left of the address bar) for the Tensorboard logger to work correctly!

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/train

## Train YOLOv5 detection model

- `--name` name of the training run
- `--img` input image size (recommended: same size as for inference)
- `--batch` determine [batch size](https://github.com/ultralytics/yolov5/issues/2377) (recommended: 32)
- `--epochs` set the number of training [epochs](https://machine-learning.paperspace.com/wiki/epoch) (recommended: 100-300+ epochs)
- `--data` path to YAML file
- `--weights` specify the pretrained model weights
> `--weights yolov5n.pt` for YOLOv5-nano model  
`--weights yolov5s.pt` for YOLOv5-small model (recommended)  
`--weights yolov5m.pt` for YOLOv5-medium model

- `--cache:` cache images for faster training

> More information on [model training](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results) 🚀

In [ ]:
training_run_name = "YOLOv5s_1class_416_batch32_epochs250" #@param {type: "string"}
#@markdown **Add UTC timestamp in front of training run name:**
add_timestamp = True #@param {type:"boolean"}
#@markdown ---
image_size = 416 #@param {type: "integer"}
batch_size = 32 #@param {type:"slider", min:16, max:128, step:16}
number_epochs = 250 #@param {type:"slider", min:10, max:600, step:10}
weights = "yolov5s.pt" #@param ["yolov5n.pt", "yolov5s.pt", "yolov5m.pt"]

if add_timestamp == True:
  from datetime import datetime
  utc_timestamp = datetime.now().strftime("%Y%m%d_%H-%M")
  train_run_name = f"{utc_timestamp}_{training_run_name}"
else:
  train_run_name = training_run_name

%cd /content/yolov5

!python train.py \
--name {train_run_name} \
--img {image_size} \
--batch {batch_size} \
--epochs {number_epochs} \
--data {dataset_location}/data.yaml \
--weights {weights} \
--cache

### View metrics plots

In [ ]:
from IPython.display import Image
Image(filename = f"/content/yolov5/runs/train/{train_run_name}/results.png", width=1000)

In [ ]:
#@title ## Export to Google Drive or Download training results {display-mode: "form"}

training_results = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving training results in Google Drive:
GDrive_save_path = "MyDrive/Training_results/YOLOv5"  #@param {type: "string"}

if training_results == "Export_Google_Drive":
  %mkdir -p /content/drive/{GDrive_save_path}
  %cp -ai /content/yolov5/runs/train/{train_run_name} /content/drive/{GDrive_save_path}
elif training_results == "Download":
  %cd /content/yolov5/runs/train
  !zip -rq {train_run_name}.zip {train_run_name}
  from google.colab import files
  files.download(f"{train_run_name}.zip")

## Validate on dataset test split

If you want to validate on the dataset valid split (default), remove the line `--task "test" \`.

In [ ]:
%cd /content/yolov5

!python val.py \
--name {train_run_name}_validate \
--weights runs/train/{train_run_name}/weights/best.pt \
--data {dataset_location}/data.yaml \
--img {image_size} \
--task "test"

In [ ]:
#@title ## Export to Google Drive or Download validation results {display-mode: "form"}

validation_results = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving validation results in Google Drive:
GDrive_save_path = "MyDrive/Training_results/YOLOv5"  #@param {type: "string"}

if validation_results == "Export_Google_Drive":
  %mkdir -p /content/drive/{GDrive_save_path}
  %cp -ai /content/yolov5/runs/val/{train_run_name}_validate /content/drive/{GDrive_save_path}
elif validation_results == "Download":
  %cd /content/yolov5/runs/val
  !zip -rq {train_run_name}_validate.zip {train_run_name}_validate
  from google.colab import files
  files.download(f"{train_run_name}_validate.zip")

## Test inference

In [ ]:
#@markdown #### Decrease confidence threshold to detect objects with lower confidence score
confidence_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}

%cd /content/yolov5

!python detect.py \
--name {train_run_name}_detect \
--weights runs/train/{train_run_name}/weights/best.pt \
--source {dataset_location}/test/images/ \
--img {image_size} \
--conf-thres {confidence_threshold} \
--line-thickness 1  # bounding box line thickness and label size (default: 3)
#--visualize        # enable feature map visualization

### Show inference results on test images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob(f"/content/yolov5/runs/detect/{train_run_name}_detect/*.jpg"):
  display(Image(filename=imageName))
  print("\n")

# Model conversion

You can upload your model weights file (**best.pt**) directly at https://tools.luxonis.com/ to automatically convert your model to .blob file.

**For manual conversion (more options) continue with the following steps.**

## Export trained model weights to ONNX format

In [ ]:
%cd /content/yolov5

!python export.py \
--weights runs/train/{train_run_name}/weights/best.pt \
--include onnx \
--simplify

### Edit outputs of the YOLOv5 ONNX model

We have to slightly edit the outputs of the YOLOv5 model for it to work with the [YoloDetectionNetwork](https://docs.luxonis.com/projects/api/en/latest/components/nodes/yolo_detection_network/) node in the DepthAI API, which was initially developed for YOLOv3 and YOLOv4.

We will cut off the last layers for post-processing, as these processing steps will be done on the OAK device during inference. The actual cutting is done in the next step when converting the model to the OpenVINO IR format, first we will define the new output layers of the ONNX model.

> More information on model cutting can be found at the [OpenVINO docs](https://docs.openvino.ai/latest/openvino_docs_MO_DG_prepare_model_convert_model_Cutting_Model.html).

In [ ]:
%pip install -q onnx
import onnx

onnx_model = onnx.load(f"/content/yolov5/runs/train/{train_run_name}/weights/best.onnx")

conv_indices = []
for i, n in enumerate(onnx_model.graph.node):
  if "Conv" in n.name:
    conv_indices.append(i)
input1, input2, input3 = conv_indices[-3:]
sigmoid1 = onnx.helper.make_node(
  'Sigmoid',
  inputs=[onnx_model.graph.node[input1].output[0]],
  outputs=['output1_yolov5'],
)
sigmoid2 = onnx.helper.make_node(
  'Sigmoid',
  inputs=[onnx_model.graph.node[input2].output[0]],
  outputs=['output2_yolov5'],
)
sigmoid3 = onnx.helper.make_node(
  'Sigmoid',
  inputs=[onnx_model.graph.node[input3].output[0]],
  outputs=['output3_yolov5'],
)
onnx_model.graph.node.append(sigmoid1)
onnx_model.graph.node.append(sigmoid2)
onnx_model.graph.node.append(sigmoid3)

onnx.save(onnx_model, f"/content/yolov5/runs/train/{train_run_name}/weights/best_cut.onnx")

In [ ]:
#@title ## Export to Google Drive or Download ONNX models {display-mode: "form"}

onnx_models = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving ONNX models in Google Drive:
GDrive_save_path = "MyDrive/Training_results/YOLOv5"  #@param {type: "string"}

%mkdir -p /content/yolov5/runs/train/{train_run_name}/weights/onnx
%cp -ai /content/yolov5/runs/train/{train_run_name}/weights/best.onnx /content/yolov5/runs/train/{train_run_name}/weights/onnx
%cp -ai /content/yolov5/runs/train/{train_run_name}/weights/best_cut.onnx /content/yolov5/runs/train/{train_run_name}/weights/onnx

if onnx_models == "Export_Google_Drive":
  %mkdir -p /content/drive/{GDrive_save_path}
  %cp -ai /content/yolov5/runs/train/{train_run_name}/weights/onnx /content/drive/{GDrive_save_path}/{train_run_name}/weights
elif onnx_models == "Download":
  %cd /content/yolov5/runs/train
  !zip -rq {train_run_name}/weights/onnx.zip {train_run_name}/weights/onnx
  from google.colab import files
  files.download(f"{train_run_name}/weights/onnx.zip")

## Convert ONNX model to OpenVINO IR format

> More information on model optimization can be found at the [OpenVINO docs](https://docs.openvino.ai/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html).

In [ ]:
#@markdown ### Name for the OpenVINO IR model:
model_name = "yolov5s_416_1class"  #@param {type: "string"}
#@markdown ---

image_size = 416 #@param {type: "integer"}

%pip install -q openvino-dev==2022.1.0

!mo \
--input_model /content/yolov5/runs/train/{train_run_name}/weights/best_cut.onnx \
--model_name {model_name} \
--output_dir /content/yolov5/runs/train/{train_run_name}/weights/openvino/ \
--input_shape [1,3,{image_size},{image_size}] \
--scale 255 \
--reverse_input_channels \
--output "output1_yolov5,output2_yolov5,output3_yolov5" \
--data_type FP16

## Convert OpenVINO IR to .blob file

> More information on blob conversion can be found at the [DepthAI docs](https://docs.luxonis.com/en/latest/pages/model_conversion/).

> More information on the number of shaves can be found at the [DepthAI FAQ](https://docs.luxonis.com/en/latest/pages/faq/#what-are-the-shaves).

In [ ]:
#@markdown ### Specify number of SHAVE cores that the converted model will use
#@markdown Recommended number of shaves: **4 - 9**
number_shaves = 9 #@param {type:"slider", min:1, max:16, step:1}

%pip install -q blobconverter boto3==1.17.39
import blobconverter

blob_path = blobconverter.from_openvino(
  xml = f"/content/yolov5/runs/train/{train_run_name}/weights/openvino/{model_name}.xml",
  bin = f"/content/yolov5/runs/train/{train_run_name}/weights/openvino/{model_name}.bin",
  data_type = "FP16",
  shaves = number_shaves,
  version = "2022.1",
  output_dir = f"/content/yolov5/runs/train/{train_run_name}/weights/openvino/blob/",
  use_cache = False
)

## Generate config JSON file

In [ ]:
#@markdown ### Name for the config JSON file:
json_name = "yolov5s_416_1class" #@param {type: "string"}
#@markdown ---

image_size = 416 #@param {type: "integer"}
number_classes = 1 #@param {type: "integer"}
#@markdown ---

#@markdown #### For several classes/labels: **["class1", "class2", "class3"]**
labels = ["insect"] #@param {type: "raw"}
#@markdown ---

#@markdown #### Decrease confidence threshold to detect objects with lower confidence score:
confidence_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
#@markdown #### Increase IoU threshold if the same object is detected multiple times:
iou_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}

masks = {
  f"side{int(image_size/8)}" : [0,1,2],
  f"side{int(image_size/16)}" : [3,4,5],
  f"side{int(image_size/32)}" : [6,7,8]
}

!wget -L https://raw.githubusercontent.com/luxonis/depthai-experiments/master/gen2-yolo/device-decoding/json/yolov5.json -P /content/

import json

with open("/content/yolov5.json", "r") as f:
  json_data = json.load(f)
  json_data["nn_config"]["input_size"] = f"{image_size}x{image_size}"
  json_data["nn_config"]["NN_specific_metadata"]["classes"] = number_classes
  json_data["nn_config"]["NN_specific_metadata"]["anchor_masks"] = masks
  json_data["nn_config"]["NN_specific_metadata"]["iou_threshold"] = iou_threshold
  json_data["nn_config"]["NN_specific_metadata"]["confidence_threshold"] = confidence_threshold
  json_data["mappings"]["labels"] = labels

with open(f"/content/yolov5/runs/train/{train_run_name}/weights/openvino/blob/{json_name}.json", "w") as f:
  json.dump(json_data, f, indent = 4)

In [ ]:
#@title ## Export to Google Drive or Download OpenVINO and blob model + config JSON {display-mode: "form"}

openvino_models = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving OpenVINO models in Google Drive:
GDrive_save_path = "MyDrive/Training_results/YOLOv5"  #@param {type: "string"}

if openvino_models == "Export_Google_Drive":
  %mkdir -p /content/drive/{GDrive_save_path}
  %cp -ai /content/yolov5/runs/train/{train_run_name}/weights/openvino /content/drive/{GDrive_save_path}/{train_run_name}/weights
elif openvino_models == "Download":
  %cd /content/yolov5/runs/train
  !zip -rq {train_run_name}/weights/openvino.zip {train_run_name}/weights/openvino
  from google.colab import files
  files.download(f"{train_run_name}/weights/openvino.zip")

# Model deployment

That's it! You trained your own YOLOv5 object detection model with your custom dataset and converted it to blob format which is necessary to run inference on the [Luxonis OAK devices](https://docs.luxonis.com/projects/hardware/en/latest/).

> To deploy the YOLOv5 model on your OAK, you can check out the Luxonis GitHub repository for [on-device decoding](https://github.com/luxonis/depthai-experiments/tree/master/gen2-yolo/device-decoding) or use the deployment options from the [Insect Detect Docs](link) (e.g. for continuous automated insect monitoring).